In [126]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import sys
from scipy import sparse

In [127]:
def create_minus_Laplace_1d(n):
    '''
    Creates 1d n×n Laplacian
    '''
    main_diag = np.full(n, 2, dtype=float)
    side_diag = np.full(n - 1, -1, dtype=float)
    diags = [main_diag, side_diag, side_diag]
    laplacian = sparse.diags(diags, [0, -1, 1], format='csr')
    return laplacian.toarray()

In [128]:
def compute_Au_1d(U):
    kernel = tf.reshape(tf.constant([-1., 2., -1.]), [3, 1, 1])
    return tf.nn.conv1d(U, kernel, stride=1, padding='SAME')

In [129]:
def frobenius_norm(A):
    return tf.sqrt(tf.reduce_sum(tf.square(A)))

In [225]:
def damped_jacobi_1d(b, init_U=None, max_iter=10, damping_factor=2./3):
    '''
    Computes damped Jacobi solution for 2d Poisson equations
    Args:
        b: A `Tensor` of (batch, n, 1) shape, 
            right-hand sides of equations
        init_U: A `Tensor` of (batch, n, 1) shape,
            initial guess for solution
        max_iter: An `int`, 
            number of iterations
    Returns:
        A `Tensor` of (batch, n, 1) shape,
            solution of 2d Poisson equation
    '''
    inv_d = 0.5
    if init_U is None:
        cur_U = tf.random_normal(tf.shape(b), mean=0, stddev=0.01)
    else:
        cur_U = tf.identity(init_U)
    kernel = tf.reshape(tf.constant([-1., 0., -1.]), [3, 1, 1])
    for i in range(max_iter):
        Ru = tf.nn.conv1d(cur_U, kernel, stride=1, padding='SAME')
        cur_U = damping_factor * inv_d * (b - Ru) + \
            (1 - damping_factor) * cur_U
    return cur_U

In [226]:
# def gauss_seidel_1d(b, init_U=None, max_iter=10):
#     n = tf.shape(b)[1]
#     range_0_n = tf.reshape(tf.range(0, n, dtype=tf.int32), shape=[-1, 1])
#     range_1_n = tf.reshape(tf.range(1, n, dtype=tf.int32), shape=[-1, 1])
#     range_0_n_minus_1 = tf.reshape(tf.range(0, n - 1, dtype=tf.int32), shape=[-1, 1])
#     indices_1 = tf.concat(1, [range_0_n, range_0_n])
#     indices_2 = tf.concat(1, [range_1_n, range_0_n_minus_1])
#     indices = tf.concat(0, [indices_1, indices_2])
#     indices = tf.to_int64(indices)
#     values = tf.concat(0, [tf.fill(tf.shape(b)[1:2], -2.), 
#                            tf.ones(tf.shape(b)[1:2] - 1)])
#     return tf.SparseTensor(indices, values, shape=(n, n))

In [227]:
def compute_R(U, R):
    '''
    Computes restriction of U
    Args:
        U: A `Tensor` of (batch, 2 ** n - 1, 1) shape,
            data to convolve
        R: A `Tensor` of (3, 1, 1) shape,
            restriction operator stencil
    Returns:
        A `Tensor` of (batch, 2 ** (n - 1) - 1, 1) shape
    '''
    return tf.nn.conv1d(U, R, stride=2, padding='VALID')

def compute_P(U, P):
    '''
    Computes prolongation of U
    Args:
        U: A `Tensor` of (batch, 2 ** n - 1, 1) shape,
            data to convolve
        P: A `Tensor` of (3, 1, 1) shape,
            projection operator stencil
    Returns:
        A `Tensor` of (batch, 2 ** (n + 1) - 1, 1) shape
    '''
    batch = tf.to_int32(tf.shape(U)[0:1])
    dim = (tf.to_int32(tf.shape(U)[1:2]) + 1) * 2 - 1
    other = tf.constant([1, 1], dtype=tf.int32)
    output_shape = tf.concat(0, [batch, dim, other])
    U = tf.reshape(U, [batch[0], -1, 1, 1])
    P = tf.reshape(P, [3, 1, 1, 1])
    Pu = tf.nn.conv2d_transpose(U, P, output_shape=output_shape, 
                                  strides=[1, 2, 2, 1], padding='VALID')
    return tf.reshape(Pu, [batch[0], -1, 1])

In [228]:
def multigrid(n_cycles, n, level, b, init_U=None, P=None, R=None, smoother_iter=2):
    '''
    Computes multigrid solution of 2d Poisson equation
    Args:
        n_cycles: An `int`, 
            number of V-cycles,
        n: An `int`,
        b and init_U have shape (batch, n, 1)
        other args are the same as in V_cycle function
    Returns:
        A `Tensor` of (batch, n, 1) shape,
        multigrid solution
    '''
    batch = tf.shape(b)[0]
    b = tf.reshape(b, (batch, n, 1))
    if init_U is None:
        init_U = tf.random_normal((tf.shape(b)), stddev=0.01)
    else:
        init_U = tf.reshape(init_U, (batch, n, 1))
    if P is None:
        P = tf.reshape((tf.constant([1., 2., 1.])) /  2, [3, 1, 1])
    if R is None:
        R = tf.reshape((tf.constant([1., 2., 1.])) /  4, [3, 1, 1])
    discrete_laplace = tf.constant(create_minus_Laplace_1d(
            2 ** int((np.log2(n + 1) - level + 1)) - 1), dtype=tf.float32)
    chol = tf.cholesky(discrete_laplace)
    for cycle_idx in range(n_cycles):
        init_U = V_cycle(level, b, chol, init_U, P, R, smoother_iter)
    return init_U

In [229]:
def V_cycle(level, b, chol, init_U=None, P=None, R=None, smoother_iter=3):
    '''
    Applies one multigrid V-cycle to Poisson 2d equation
    Args:
        level: An `int`, 
            number of multigrid levels
        b: A `Tensor` of (batch, n, 1) shape, 
            right-hand sides of equations
        chol: A `Tensor` of (n, n) shape,
            laplacian for the last MG level cholesky L
        init_U: A `Tensor` of (batch, n, 1) shape or None,
            initial guess for solution, 
            if init_U is None, it will be random
        P: A `Tensor` variable of (3, 1, 1) shape,Или с семи, чтобы же
            prolongation operator stencil
        R: A `Tensor` variable of (3, 1, 1) shape,
            restriction operator stencil
        smoother_iter: An `int`,
            number of smoother iterations
    Returns:
        A `Tensor` of (batch, n, 1) shape,
        multigrid solution
    '''
    if init_U is None:
        init_U = tf.random_normal((tf.shape(b)), stddev=0.01)
    if level == 1:
        multiples = tf.concat(0, [tf.shape(b)[0:1], tf.constant([1, 1])])
        tiled_chol = tf.tile(tf.expand_dims(chol, axis=0), multiples)
        new_U = tf.cholesky_solve(tiled_chol, b)
    else:
        U = damped_jacobi_1d(b, init_U, max_iter=smoother_iter)
        Au = compute_Au_1d(U)
        res = compute_R(b - Au, R)
        e = tf.zeros_like(res)
        e = V_cycle(level - 1, b=res, chol=chol, init_U=e , P=P, R=R, 
                    smoother_iter=smoother_iter)
        e = tf.reshape(e, tf.shape(res))
        new_U = U + compute_P(e, P)
        new_U = damped_jacobi_1d(b, new_U, max_iter=smoother_iter)
    return new_U

In [230]:
n = 15
P = tf.reshape(tf.constant([1., 2., 1.]) / 2, (3, 1, 1))
R = tf.identity(P) / 2
batch = 1
b = tf.ones((batch, n, 1))
U = tf.zeros((batch, n, 1))
multigrid_U = multigrid(n_cycles=1, level=2, n=n, b=b)
res = frobenius_norm(b - compute_Au_1d(multigrid_U))
with tf.Session() as sess:
    #sol = sess.run(multigrid_U)
    np_res = sess.run(res)

In [231]:
n = 15
b = tf.ones((batch, n, 1))
U = tf.zeros((batch, n, 1))
res_arr = []
sol_arr = []
for n_iter in range(20):
    res_arr.append(frobenius_norm(compute_Au_1d(U) - b) / frobenius_norm(b))
    U = multigrid(n_cycles=1, n=n, level=2, b=b, smoother_iter=2, P=P, R=R, init_U=U)
    sol_arr.append(U)
res_arr.append(frobenius_norm(compute_Au_1d(U) - b) / frobenius_norm(b))
with tf.Session() as sess:
    np_res = np.array(sess.run(res_arr))
    np_sol = sess.run(sol_arr)

In [232]:
np_res[1:] / np_res[:-1]

array([ 0.67905724,  0.70483512,  0.71042496,  0.71272713,  0.71375304,
        0.71421492,  0.71442205,  0.71451622,  0.71455574,  0.71457273,
        0.71458685,  0.71458602,  0.71456534,  0.71460563,  0.71460378,
        0.71459621,  0.71447265,  0.71460599,  0.71469879,  0.71455008], dtype=float32)